<a href="https://colab.research.google.com/github/gomlfx/apiaryfund/blob/master/RandomForest_CLF_RGR_risk_reward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install autosklearn
#!pip install pandas_ta

In [2]:
#import autosklearn
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb
import numpy as np
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
#more testing
from sklearn.naive_bayes import GaussianNB

In [4]:
df = pd.read_csv("EURUSD_Daily_197101040000_202009110000.csv", sep='\t')
print(df.head(1))
print(df.tail(1))

       <DATE>  <OPEN>  <HIGH>   <LOW>  <CLOSE>  <TICKVOL>  <VOL>  <SPREAD>
0  1971.01.04  0.5369  0.5369  0.5369   0.5369          1      0        50
           <DATE>   <OPEN>   <HIGH>  ...  <TICKVOL>  <VOL>  <SPREAD>
12801  2020.09.11  1.18145  1.18739  ...      46151      0         1

[1 rows x 8 columns]


In [5]:
#from pandas import to_datetime
#df['<DATE>'] = to_datetime(df['<DATE>'])
#df.tail()
df['<DATE>'] = df['<DATE>'].str.replace('.', '')

In [6]:
df['shifted_high'] = df['<HIGH>'].shift(periods=-1)
df['shifted_close'] = df['<CLOSE>'].shift(periods=-1)
df['shifted_low'] = df['<LOW>'].shift(periods=-1)

df = df.rename(columns={"<DATE>":"date",'<OPEN>':'open','<HIGH>':'high','<LOW>':'low','<CLOSE>':'close','<TICKVOL>':'tickvol','<VOL>':'volume'})
df.tail(-2)

,date,open,high,low,close,tickvol,volume,<SPREAD>,shifted_high,shifted_close,shifted_low
2,19710106,0.53650,0.53650,0.53650,0.53650,1,0,50,0.53680,0.53680,0.53680
3,19710107,0.53680,0.53680,0.53680,0.53680,1,0,50,0.53710,0.53710,0.53710
4,19710108,0.53710,0.53710,0.53710,0.53710,1,0,50,0.53710,0.53710,0.53710
5,19710111,0.53710,0.53710,0.53710,0.53710,1,0,50,0.53710,0.53710,0.53710
6,19710112,0.53710,0.53710,0.53710,0.53710,1,0,50,0.53730,0.53730,0.53730
...,...,...,...,...,...,...,...,...,...,...,...
12797,20200907,1.18458,1.18485,1.18111,1.18136,31385,0,1,1.18273,1.17739,1.17654
12798,20200908,1.18136,1.18273,1.17654,1.17739,56953,0,1,1.18337,1.18027,1.17528
12799,20200909,1.17747,1.18337,1.17528,1.18027,50318,0,1,1.19174,1.18146,1.17994
12800,20200910,1.18026,1.19174,1.17994,1.18146,60593,0,1,1.18739,1.18459,1.18086


In [7]:
#standardize: o-c, o-h, o-l
df['st_high'] = df['high'] - df['open']
df['st_close'] = df['close'] - df['open']
df['st_low'] = df['low'] - df['open']

#to try to standardize
df['st_open'] = 0 

df['sh_st_high'] = df['st_high'].shift(periods=-1)
df['sh_st_close'] = df['st_close'].shift(periods=-1)
df['sh_st_low'] = df['st_low'].shift(periods=-1)




In [8]:
# from sklearn.preprocessing import MinMaxScaler
# #scale the training df
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(df)
# scaled_train = pd.DataFrame(scaled)
# scaled_train

In [9]:
#current biggest extrema is high(1) or low(0)
conditions = [
              (df['st_high'] > df['st_low'].multiply(-1)),
              (df['st_high'] < df['st_low'].multiply(-1)),
             ]
values = [1,0]
df['current_ext'] = np.select(conditions,values)

#predicted biggest extrema is high(1) or low(0)
conditions = [
              (df['sh_st_high'] > df['sh_st_low'].multiply(-1)),
              (df['sh_st_high'] < df['sh_st_low'].multiply(-1)),
             ]
values = [1,0]
df['prob_sh_ext'] = np.select(conditions,values)

In [10]:
df = df.head(-1)
df.tail()

,date,open,high,low,close,tickvol,volume,<SPREAD>,shifted_high,shifted_close,shifted_low,st_high,st_close,st_low,st_open,sh_st_high,sh_st_close,sh_st_low,current_ext,prob_sh_ext
12796,20200904,1.18513,1.18652,1.17808,1.18394,58184,0,1,1.18485,1.18136,1.18111,0.00139,-0.00119,-0.00705,0,0.00027,-0.00322,-0.00347,0,0
12797,20200907,1.18458,1.18485,1.18111,1.18136,31385,0,1,1.18273,1.17739,1.17654,0.00027,-0.00322,-0.00347,0,0.00137,-0.00397,-0.00482,0,0
12798,20200908,1.18136,1.18273,1.17654,1.17739,56953,0,1,1.18337,1.18027,1.17528,0.00137,-0.00397,-0.00482,0,0.00590,0.00280,-0.00219,0,1
12799,20200909,1.17747,1.18337,1.17528,1.18027,50318,0,1,1.19174,1.18146,1.17994,0.00590,0.00280,-0.00219,0,0.01148,0.00120,-0.00032,1,1
12800,20200910,1.18026,1.19174,1.17994,1.18146,60593,0,1,1.18739,1.18459,1.18086,0.01148,0.00120,-0.00032,0,0.00594,0.00314,-0.00059,1,1


In [11]:
#find which extrema is predicted to hit for the day
clf = RandomForestClassifier(n_estimators=1000)

features = ['open','high','low','close','current_ext']
target = 'prob_sh_ext'
train = df

clf.fit(train[features],train[target])
print('For risk:reward predicted extrema:')
predicted = clf.predict([[1.18145,1.18739,1.18086,1.18459,1]]) 
print(predicted)

For risk:reward predicted extrema:
[1]


In [12]:
#fit df to model, find exact value for classifier prediction
model = RandomForestRegressor(n_estimators=1000)
features = ['open','high','low','close','current_ext']
target1 = 'shifted_high'
target2 = 'shifted_low'
target3 = 'shifted_close'
train = df

model.fit(train[features],train[target1])
print('For risk:reward predicted high:')
predicted = model.predict([[1.18145,1.18739,1.18086,1.18459,1]]) 
print(predicted)

model.fit(train[features],train[target3])
print('For risk:reward predicted close:')
predicted = model.predict([[1.18145,1.18739,1.18086,1.18459,1]]) 
print(predicted)

model.fit(train[features],train[target2])
print('For risk:reward predicted low:')
predicted = model.predict([[1.18145,1.18739,1.18086,1.18459,1]]) 
print(predicted)



For risk:reward predicted high:
[1.19071743]
For risk:reward predicted close:
[1.18395761]
For risk:reward predicted low:
[1.18086848]
